충남대학교 컴퓨터공학과 2018 2학기 인공지능 수업 과제 201504278 강천성

In [1]:
import os
from os.path import join

import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
inputPath = join("./","DataSet")
inputPath

'./DataSet'

In [3]:
os.listdir(inputPath)

['iris.csv']

In [4]:
# 데이터 불러들이기
iris = pd.read_csv(join(inputPath,'iris.csv'), names=("sepal length", "sepal width", "petal length", "petal width", "Class"))
iris.head()

,sepal length,sepal width,petal length,petal width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
#클래스 이름 확인
classes = iris["Class"].unique()

In [6]:
# 클래스 이름을 Iris-setosa : 1, Iris-versicolor : 2, Iris-virginica : 3 으로 대체
iris["Class"].replace(classes[0], 1, inplace = True) 
iris["Class"].replace(classes[1], 2, inplace = True) 
iris["Class"].replace(classes[2], 3, inplace = True) 

# 복원 추출로 10개를 샘플링해 제대로 클래스 이름이 숫자로 대체 되었는지 확인.
iris.sample(10)

,sepal length,sepal width,petal length,petal width,Class
81,5.5,2.4,3.7,1.0,2
122,7.7,2.8,6.7,2.0,3
71,6.1,2.8,4.0,1.3,2
31,5.4,3.4,1.5,0.4,1
119,6.0,2.2,5.0,1.5,3
129,7.2,3.0,5.8,1.6,3
114,5.8,2.8,5.1,2.4,3
135,7.7,3.0,6.1,2.3,3
89,5.5,2.5,4.0,1.3,2
61,5.9,3.0,4.2,1.5,2


One vs Rest Multiclass Classification https://en.wikipedia.org/wiki/Multiclass_classification#One-vs.-rest

선형 분리로 One vs Rest 다중 클래스 분류를 할것이기 때문에 SVM Classifier의 Kernel 중 LinearKernel을 사용합니다.

In [7]:
iris_x = deepcopy(iris)
iris_y = deepcopy(iris['Class'])
del iris_x['Class']

In [8]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(iris_x,iris_y, test_size=0.20)
# 훈련데이터와 테스트 데이터로 분리


In [9]:
# 분류기에 넣을 데이터 준비. 1번 분류기는 1번 클래스에 대해서만 True로 판별하고 2,3 번 클래스에 대해서는 False로 판별해야 하므로,
# 1번 클래스만 1로 2,3번 클래스는 0으로 변경하여 데이터를 넣어줄 예정. 2번 3번 분류기에 대해서도 동일하게 데이터를 넣어주려고 한다.
x_train = list()
y_train = list()
x_test = list()
y_test = list()
for i in range(len(classes)):
    x_train.append(deepcopy(train_x))
    y_train.append(deepcopy(train_y))
    x_test.append(deepcopy(test_x))
    y_test.append(deepcopy(test_y))
    if i == 0:
        y_train[i].replace(i+1,1, inplace = True)
        y_train[i].replace(2,-1, inplace = True)
        y_train[i].replace(3,-1, inplace = True)
        y_test[i].replace(i+1,1, inplace = True)
        y_test[i].replace(2,-1, inplace = True)
        y_test[i].replace(3,-1, inplace = True)
    elif i == 1:
        y_train[i].replace(1,-1, inplace = True)
        y_train[i].replace(i+1,1, inplace = True)
        y_train[i].replace(3,-1, inplace = True)
        y_test[i].replace(1,-1, inplace = True)
        y_test[i].replace(i+1,1, inplace = True)
        y_test[i].replace(3,-1, inplace = True)
    else :
        y_train[i].replace(1,-1, inplace = True)
        y_train[i].replace(2,-1, inplace = True)
        y_train[i].replace(i+1,1, inplace = True)
        y_test[i].replace(1,-1, inplace = True)
        y_test[i].replace(2,-1, inplace = True)
        y_test[i].replace(i+1,1, inplace = True)
# print(y_test[2])
for i in range(3):
    x_train[i].astype(np.float64)
    y_train[i].astype(np.float64)
    x_test[i].astype(np.float64)
    y_test[i].astype(np.float64)

In [10]:
weight_1 = np.random.normal(0, 1, 4)
weight_2 = np.random.normal(0, 1, 4)
weight_3 = np.random.normal(0, 1, 4)

bias_1 = np.random.normal(0,1,1)
bias_2 = np.random.normal(0,1,1)
bias_3 = np.random.normal(0,1,1)

# train iteration 500
for _ in range(500) : 
    # Class 1
    for x, y in zip(x_train[0].values, y_train[0].values) :
        result_1 = np.dot(x, weight_1) + bias_1
        result_1 = np.sign(result_1)
        if(result_1 != y):
            weight_1 = weight_1 + np.dot(y, x)
        
    # Class 2
    for x, y in zip(x_train[1].values, y_train[1].values) :
        result_2 = np.dot(x, weight_2) + bias_2
        result_2 = np.sign(result_2)
        if(result_2 != y):
            weight_2 = weight_2 + np.dot(y, x)
    # Class 3
    for x, y in zip(x_train[2].values, y_train[2].values) :
        result_3 = np.dot(x, weight_3) + bias_3
        result_3 = np.sign(result_3)
        if(result_3 != y):
            weight_3 = weight_3 + np.dot(y, x)

In [11]:
result_1 = np.dot(x_test[0].values, weight_1) + bias_1
result_1 = np.sign(result_1)

result_2 = np.dot(x_test[1].values, weight_2) + bias_2
result_2 = np.sign(result_2)

result_3 = np.dot(x_test[2].values, weight_3) + bias_3
result_3 = np.sign(result_3)

accuracy_1 = sum([ 1 if predict_y == true_y else 0 for predict_y, true_y in zip(result_1, y_test[0])])/len(y_test[0])
accuracy_2 = sum([ 1 if predict_y == true_y else 0 for predict_y, true_y in zip(result_2, y_test[1])])/len(y_test[1])
accuracy_3 = sum([ 1 if predict_y == true_y else 0 for predict_y, true_y in zip(result_3, y_test[2])])/len(y_test[2])


print("Class 1만 분류하는 분류기의 정확도 : "+str('{:.2f}').format(accuracy_1*100) + " Class 2만 분류하는 분류기의 정확도 : " 
      + str('{:.2f}').format(accuracy_2*100) +" Class 3만 분류하는 분류기의 정확도 : "+ str('{:.2f}').format(accuracy_3*100))

Class 1만 분류하는 분류기의 정확도 : 100.00 Class 2만 분류하는 분류기의 정확도 : 86.67 Class 3만 분류하는 분류기의 정확도 : 96.67


In [12]:
predict_y = list()

# 3개의 선형 분류기를 합쳐서 1개의 다중 분류기 만들기
for x in test_x.values:
    result_1 = np.dot(x, weight_1) + bias_1
    result_1 = np.sign(result_1)

    result_2 = np.dot(x, weight_2) + bias_2
    result_2 = np.sign(result_2)

    result_3 = np.dot(x, weight_3) + bias_3
    result_3 = np.sign(result_3)
    
    if result_1 == 1 :
        predict_y.append(1)
    elif result_2 == 1:
        predict_y.append(2)
    elif result_3 == 1:
        predict_y.append(3)
    else :
        # 3개의 분류기가 분류하지 못했다면, 3개의 직선으로 표현되지 못하는 부분의 데이터라고 생각해볼 수 있다.
        predict_y.append(0)

In [13]:
# test_y 와 예측한 y의 값을 비교해 정확도를 얻어낸다.
accuracy = sum([ 1 if predict_y == true_y else 0 for predict_y, true_y in zip(predict_y, test_y)])/len(test_y)

print(str('3개의 Single Perceptron을 사용한 MultiClass Classifier의 정확도 : {:.2f}').format(accuracy*100))

3개의 Single Perceptron을 사용한 MultiClass Classifier의 정확도 : 86.67


생각보다 정확도가 잘 안나왔다. 아무래도 3개의 직선으로만 표현하려다 보니 가운데에 위치하거나 특정 위치의 값을 잘 분류하지 못하는 것 같다.